# **Construcción automática de texto (Pytorch-Ligthning)**
Andrey Duvan Rincon Torres

---

In [ ]:
pip install pytorch-lightning

In [ ]:
pip install pyyaml==5.4.1

In [ ]:
pip install plotly_express

In [ ]:
pip install torchmetrics

In [5]:
# Basicas
import pandas as pd
import numpy as np
# Graficas
import plotly.express as plx
import plotly.graph_objects as go
# Pytorch
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchmetrics.functional import accuracy
# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
# Texto
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
def poem_to_string(poem):
  return f'\n{poem["title"]}\n{poem["author"]}\n{poem["content"]}'
def poem_sequence_to_string(poem_sequence):
    poem_stringified = tokenizer.sequences_to_texts([poem_sequence])[0]
    print(poem_stringified)
CELoss = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Datos**

In [6]:
url = 'https://raw.githubusercontent.com/andreamorgar/poesIA/master/data/poems.csv'
poems_df = pd.read_csv(url)
poems_df = poems_df.dropna()

In [7]:
# Filtrar poemas grandes
poems_df['string'] = poems_df.apply(lambda row: f'\n{row["title"]}\n\n{row["author"]}\n\n{row["content"]}', axis=1)
poems_df['length'] = poems_df.string.map(len)
MAX_POEM_LENGTH=1000
poems_filtered = poems_df[poems_df.length<MAX_POEM_LENGTH]
poems_df

,author,content,title,string,length
0,Leopoldo Lugones,\n\nEn el parque confuso\nQue con lánguidas br...,LA MUERTE DE LA LUNA,\nLA MUERTE DE LA LUNA\n\nLeopoldo Lugones\n\n...,1850
1,Marilina Rébora,"\n\nPorque si tú no velas, vendré como ladrón;...",PORQUE SI TÚ NO VELAS,\nPORQUE SI TÚ NO VELAS\n\nMarilina Rébora\n\n...,732
2,Antonio Colinas,"\n\nPequeña de mis sueños, por tu piel las pal...",POEMA DE LA BELLEZA CAUTIVA QUE PERDÍ,\nPOEMA DE LA BELLEZA CAUTIVA QUE PERDÍ\n\nAnt...,662
3,José María Hinojosa,\n\nLos dedos de la nieve\nrepiquetearon\nen e...,SENCILLEZ,\nSENCILLEZ\n\nJosé María Hinojosa\n\n\n\nLos ...,273
4,Rubén Izaguirre Fiallos,"Naciste en Armenia,\npero te fuiste a vivir al...",Breve Carta a Consuelo Suncín,\nBreve Carta a Consuelo Suncín\n\nRubén Izagu...,416
...,...,...,...,...,...
5128,Rubén Darío,"\n¿Vienes? Me llega aquí, pues que suspiras, \...",Divagación,\nDivagación\n\nRubén Darío\n\n\n¿Vienes? Me l...,5157
5129,David Escobar Galindo,\n\nNada es memoria: todo es invención.\nLo qu...,Nada es memoria,\nNada es memoria\n\nDavid Escobar Galindo\n\n...,258
5130,amistad,\nFelicidad: Muy dentro de tí.\nSerenidad: En ...,Esto es todo lo que deseo para tí,\nEsto es todo lo que deseo para tí\n\namistad...,504
5131,Octavio Paz,\nMis manos \nabren las cortinas de tu ser \nt...,Palpar,\nPalpar\n\nOctavio Paz\n\n\nMis manos \nabren...,173


In [8]:
print(poems_df['string'][0])


LA MUERTE DE LA LUNA

Leopoldo Lugones



En el parque confuso
Que con lánguidas brisas el cielo sahúma,
El ciprés, como un huso,
Devana un ovillo de de bruma.
El telar de la luna tiende en plata su urdimbre;
Abandona la rada un lúgubre corsario,
Y después suena un timbre
En el vecindario.

Sobre el horizonte malva
De una mar argentina,
En curva de frente calva
La luna se inclina,
O bien un vago nácar disemina
Como la valva
De una madreperla a flor del agua marina.

Un brillo de lóbrego frasco
Adquiere cada ola,
Y la noche cual enorme peñasco
Va quedándose inmensamente sola.

Forma el tic-tac de un reloj accesorio,
La tela de la vida, cual siniestro pespunte.
Flota en la noche de blancor mortuorio
Una benzoica insispidez de sanatorio,
Y cada transeúnte
Parece una silueta del Purgatorio.

Con emoción prosaica,
Suena lejos, en canto de lúgubre alarde,
Una voz de hombre desgraciado, en que arde
El calor negro del rom de Jamaica.
Y reina en el espíritu con subconsciencie arcaica,
El miedo

# **Vocabulary**

In [9]:
poems_string=poems_filtered.string
STOP_SIGN = '␣'
tokenizer = Tokenizer(char_level=True, filters='', lower=False, split='')
tokenizer.fit_on_texts([STOP_SIGN])
tokenizer.fit_on_texts(poems_string)
total_words = len(tokenizer.word_index) + 1
dataset_vectorized = tokenizer.texts_to_sequences(poems_string)

In [18]:
input_sequences = []
for line in dataset_vectorized:
  token_list = line
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [20]:
max_sequence_len = max([len(x) for x in input_sequences])

In [ ]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# crea predictores y etiqueta
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

## **Modelo**

In [ ]:
class Model(nn.Module):
  # creamos la estructura de la red
  def __init__(self):
      super(Model,self).__init__()
      # Embeding de las palabras
      self.embedding = nn.Embedding(total_words, 100)
      # Red Bidireccional
      self.lstm_1 = nn.LSTM(100, 150, 1, batch_first=True, dropout = 0.2 , bidirectional = True)
      # red LSTM
      self.lstm_2 = nn.LSTM(300*(max_sequence_len - 1),100,1, batch_first=True)
      # red perceptron
      self.linear_1 = nn.Linear(100,1605)
      self.linear_2 = nn.Linear(1605,total_words)
      self.relu = nn.ReLU()
      self.sofmax = nn.Softmax()
  # definimos el comportamiento de las capas
  def forward(self, x):
      batch_size, channelsn = x.size()
      x = self.embedding(x)
      # layer LSTM bidirectional
      out, (h_n, c_n) = self.lstm_1(x)
      # layer LSTM
      out, (h_n, c_n) = self.lstm_2(out.reshape((batch_size,-1)))
      # capa de salida
      out = self.relu(self.linear_1(out))
      out = self.sofmax(self.linear_2(out))
      return out

## **Datos**

In [ ]:
# Clase de los datos
class DataModule(pl.LightningDataModule):
  # Definimos un tamaño de lote en la calse
  def __init__(self, batch_size = 32):
      super(DataModule,self).__init__()
      self.batch_size = batch_size
  # Definimos el tratamiento de los datos
  def setup(self, stage=None):
    x, y = predictors, label
    # Conjunto de validacion
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.1, random_state = 0,shuffle=True)
    # Transformar en tensores
    self.train_dataset = TensorDataset(torch.tensor(x_train.astype(np.int32)),torch.tensor(y_train.astype(np.int32)))
    self.val_dataset = TensorDataset(torch.tensor(x_val.astype(np.int32)),torch.tensor(y_val.astype(np.int32)))
  # Iterable de entrenamiento
  def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch_size)
  # Iterable de validacion
  def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch_size)

## **Entrenamiento**

In [ ]:
class Train(pl.LightningModule):
    # creamos la estructura de la red
    def __init__(self,model):
        super().__init__()
        self.model = model
   # Paso de entrenamiento
    def training_step(self, batch, batch_idx):
        loss,acc = self._shared_eval_step(batch, batch_idx)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss
    # Paso de validacion
    def validation_step(self, batch, batch_idx):
        loss,acc = self._shared_eval_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss
    # Funcion para evaluar el modelo y la perdida
    def _shared_eval_step(self,batch,batch_idx):
        x, y  = batch
        y_hat = self.model(x)
        loss = CELoss(y_hat, y.type(torch.LongTensor))
        acc = accuracy(y_hat, y.type(torch.LongTensor))
        return loss, acc
    # Configuracion del optimizador
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters())

##  Ajustar el modelo

In [ ]:
data_module = DataModule() # Ejecutamos modulo de datos
torch.manual_seed(0)
model = Model() # Ejecutamos modelo
trainer = pl.Trainer(max_epochs=200, progress_bar_refresh_rate=20) # Lamamos el entrenador
task = Train(model)
trainer.fit(task,data_module)

In [ ]:
# Tablero de resultados
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

# **Generacion de texto**

In [ ]:
seed_text = "Who are you, so too cruel?"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.forward(torch.tensor(token_list.astype(np.float32))).detach().numpy().transpose()[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)